In [17]:
import pandas as pd
import numpy as np

df = pd.read_excel(r"C:\Users\lawsonsa\Downloads\01-25-2022 Picklist of 44 miRNAs (modified).xlsx", sheet_name = 'Sort by plate', index_col='Sample Group')
target = pd.DataFrame(columns=['Source Plate Barcode ', 'Sample ID', 'Source Plate Name',
       'Source well', 'Row', 'Col', 'Sample Name', 'Sample Group',
       'Sample Comment', 'Destination well', 'Destination Plate Barcode',
       'Destination Volume'])


target
df.columns

Index(['Source Plate Barcode ', 'Sample ID', 'Source Plate Name',
       'Source well', 'Row', 'Col', 'Sample Name', 'Full Gene Name', 'Gene ID',
       'siRNA ID', 'Unnamed: 11'],
      dtype='object')

In [22]:
def sort_by_frequency(df, column):
    '''
    Sort a pandas dataframe based on the frequency of a provided column in that pandas dataframe.
    i.e. if the kwarg column is plates, we want to be able to sort the dataframe so that the rows with the least common plate appears at the top, and the most common plate appears at the end.
    '''
    # Get the frequency of each label in the column
    frequencies = df[column].value_counts()
    print(f'The abundance of {column} is:\n{frequencies}')
    
    # Make a new column with the frequencies
    df['frequency'] = df[column].apply(lambda x: frequencies[x])
    
    # Sort the dataframe by the frequency column
    #note that we also sort alphabetically on the kwarg AFTER sorting by frequency, so that we don't get tripped up by any ties in frequency
    df.sort_values(by=['frequency', column], inplace=True) 
    # Drop the frequency column
    df = df.drop(columns='frequency')
    return df

df = sort_by_frequency(df, 'Source Plate Name')
allmirs = df.index
allmirs

The abundance of Source Plate Name is
Hm Mimic v21-1    29
Hm Mimic v21-2    12
Hm Mimic v21-4     2
Hm Mimic v21-3     1
Name: Source Plate Name, dtype: int64


Index(['hsa-miR-2110', 'hsa-miR-3714', 'hsa-miR-3937', 'hsa-miR-17-5p',
       'hsa-miR-18a-5p', 'hsa-miR-106a-5p', 'hsa-miR-135b-5p',
       'hsa-miR-143-5p', 'hsa-miR-193b-3p', 'hsa-miR-200c-3p',
       'hsa-miR-340-5p', 'hsa-miR-450b-3p', 'hsa-miR-452-5p', 'hsa-miR-873-5p',
       'hsa-miR-128-3p', 'hsa-miR-10a-5p', 'hsa-miR-10b-5p', 'sample group',
       'hsa-miR-19a-3p', 'hsa-miR-19b-3p', 'hsa-miR-20a-5p', 'hsa-miR-20b-5p',
       'hsa-miR-25-3p', 'hsa-miR-27a-3p', 'hsa-miR-27b-3p', 'hsa-miR-34a-5p',
       'hsa-miR-34b-5p', 'hsa-miR-92a-3p', 'hsa-miR-93-5p', 'hsa-miR-103a-3p',
       'hsa-miR-106b-5p', 'hsa-miR-107', 'hsa-miR-124-3p', 'hsa-miR-137',
       'hsa-miR-196a-5p', 'hsa-miR-200a-3p', 'hsa-miR-200b-3p',
       'hsa-miR-204-5p', 'hsa-miR-211-5p', 'hsa-miR-363-3p', 'hsa-miR-429',
       'hsa-miR-449a', 'hsa-miR-449b-5p', 'hsa-miR-506-3p'],
      dtype='object', name='Sample Group')

In [24]:
def get_pairs(series):
    pairs = []
    for i in range(len(series)):
        for j in range(len(series)):
            if (series[j], series[i]) not in pairs:
                pairs.append((series[i], series[j]))
    return pairs

pairs = get_pairs(allmirs) #create list of tuples of nonredundant pairs + solos
len(pairs)

990

In [25]:
rows = ['B','C','D','E','F','G','H','I','J','K','L','M','N','O']
cols = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
wells = [row+str(col) for row in rows for col in cols]

In [27]:
import math

variable1 = len(wells) #num wells available per plate
variable2 = len(pairs) #num wells needed
variable3 = int(math.ceil(variable2 / variable1)) #num plates needed
variable4 = (variable1*variable3)-variable2
print(f'Num wells per plate: {variable1}\nNum wells needed for screen: {variable2}\nNum plates needed for screen: {variable3}\nSpare wells: {variable4}')

Num wells per plate: 252
Num wells needed for screen: 990
Num plates needed for screen: 4
Spare wells: 18


In [7]:
for plate in range(variable3):
    for well in wells:
        if len(pairs) !=0:
            target = target.append(pd.concat([df.loc[pairs[0][0]][:7], pd.Series({'Sample Group' : pairs[0][0], 'Sample Comment': '', 'Destination well':well, 'Destination Plate Barcode':plate+1, 'Destination Volume':''})]), ignore_index=True)
            target = target.append(pd.concat([df.loc[pairs[0][1]][:7], pd.Series({'Sample Group' : pairs[0][1], 'Sample Comment': '', 'Destination well':well, 'Destination Plate Barcode':plate+1, 'Destination Volume':''})]), ignore_index=True)
            pairs.pop(0)

In [8]:
target.to_csv('pairwise_echo_instructions.csv")
target

,Source Plate Barcode,Sample ID,Source Plate Name,Source well,Row,Col,Sample Name,Sample Group,Sample Comment,Destination well,Destination Plate Barcode,Destination Volume
0,CPE200EL,NaN,Hm Mimic v21-4,G21,G,21,MIMAT0018352,hsa-miR-3937,,B4,1,
1,CPE200EL,NaN,Hm Mimic v21-4,G21,G,21,MIMAT0018352,hsa-miR-3937,,B4,1,
2,CPE200EL,NaN,Hm Mimic v21-4,G21,G,21,MIMAT0018352,hsa-miR-3937,,B5,1,
3,CPE200EL,NaN,Hm Mimic v21-4,L9,L,9,MIMAT0018165,hsa-miR-3714,,B5,1,
4,CPE200EL,NaN,Hm Mimic v21-4,G21,G,21,MIMAT0018352,hsa-miR-3937,,B6,1,
...,...,...,...,...,...,...,...,...,...,...,...,...
1975,CPE200F7,ASO21XZ5,Hm Mimic v21-1,E14,E,14,MIMAT0000254,hsa-miR-10b-5p,,N19,4,
1976,CPE200F7,ASO21XZ5,Hm Mimic v21-1,E14,E,14,MIMAT0000254,hsa-miR-10b-5p,,N20,4,
1977,CPE200F7,ASO21XXQ,Hm Mimic v21-1,L21,L,21,MIMAT0000253,hsa-miR-10a-5p,,N20,4,
1978,CPE200F7,ASO21XXQ,Hm Mimic v21-1,L21,L,21,MIMAT0000253,hsa-miR-10a-5p,,N21,4,
